In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import mplcursors
import seaborn as sns
import umap
import ipympl
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from math import sin, cos, sqrt, atan2, radians
from ipywidgets import Checkbox, HBox, VBox, Layout, Button, FloatSlider, FloatText, IntText, HTML
from IPython.display import Javascript, display
from functools import partial
from colormath.color_objects import LabColor, sRGBColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000

# approximate radius of earth in km
R = 6373.0
# Should use this for measuring distances if possible
def lldist(lat1, lon1, lat2, lon2):
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

df = pd.read_excel("Grilla para Urbest 2020-08-26.xlsx",sheet_name=0).dropna()
desc = pd.read_excel("Grilla para Urbest 2020-08-26.xlsx",sheet_name=1).dropna(subset=['Variable a 800 metros'])
#print(df.head)
%matplotlib widget 
#All graphs are set to interactive to avoid issues when switching matplotlib graphing modes

### Variables selection

In [27]:
#Extracting the variables from the metadata sheet, along descriptions
desc_main =  {var: desc.loc[desc['Variable a 800 metros'] == var, 'Definición'].values[0] for var in desc['Variable a 800 metros'].to_list()[:1]}
#hm17com_pob_tot is neither a variable at 800m nor at 1600m, it is added first to the variables list
desc_main.update({'latitude':'Latitud','longitude':'Longitud'})
desc_800 = {var: desc.loc[desc['Variable a 800 metros'] == var, 'Definición'].values[0] + ' (800 metros)' for var in desc['Variable a 800 metros'].to_list()[1:]}
desc_1600 = {var: desc.loc[desc['Variable a 1600 metros'] == var, 'Definición'].values[0] + ' (1600 metros)' for var in desc['Variable a 1600 metros'].to_list()[1:]}
desc_all = {} 
desc_all.update(desc_main)
desc_all.update(desc_800)
desc_all.update(desc_1600)
#By default, geo variables, aggregate variables and variables at 1600m are disabled
bools_all = {i: False if '(1600 metros)' in desc_all[i] else True for i in desc_all}
bools_all['hm17mz_viv_tot_ssum2'] = False
bools_all['hm17mz_gse_thog_ssum2'] = False
bools_all['hm17mz_gse_alt_ssum2'] = False
bools_all['hm17mz_gse_med_ssum2'] = False
bools_all['hm17mz_gse_baj_ssum2'] = False
bools_all['pore2_HospitalesyClínicas'] = False
bools_all['latitude'] = False
bools_all['longitude'] = False
#print(desc_all)

{'hm17com_pob_tot': 'Población a nivel comunal, no está definida por distancia al punto, sino por la comuna donde cae'}


In [3]:
#Build variables checkbox list from extracted variable metadata
ev_count = 0
def tick(ch,key):
    global ev_count    #Checkbox widget activates twice per click. Avoid that
    ev_count += 1
    if ev_count > 2:
        val = ch.owner.value
        bools_all[key] = val
        #print(bools_all)
        ev_count = 0
    return 
chboxes = [HBox([Checkbox(bools_all[i],description=i),HTML(value='<style>p{word-wrap: break-word}</style><p>'+desc_all[i]+'</p>')]) for i in desc_all.keys()] #Add variable name to description...
ch_per_wcol = int(len(chboxes)/3)
wcol_1 = VBox(chboxes[:ch_per_wcol])
wcol_2 = VBox(chboxes[ch_per_wcol:ch_per_wcol*2])
wcol_3 = VBox(chboxes[ch_per_wcol*2:])
hb = HBox([wcol_1,wcol_2,wcol_3])
for ch in chboxes:
    k = ch.children[0].description
    ch.children[0].description = "" #...then remove variable name after it has been assigned to the event handler
    ch.observe(partial(tick,key=k))
display(hb)

In [4]:
#Make model widget. Currently just runs all cells below
def crear_modelo(btn):
    display(Javascript('IPython.notebook.execute_cells_below()'))
#TODO: Encapsulate methods in functions so it doesn't execute all cells. Though it may still need to
btn = Button(description="Make model")
btn.on_click(crear_modelo)
display(btn)

Button(description='Make model', style=ButtonStyle())

In [5]:
#Get the selected variables list from checkboxes
#Not part of the variables set: 'id','name','project','status','city',
data_cols = [i for i in bools_all if bools_all[i] == True]
#print(data_cols)

In [6]:
#Calculating the UMAP model
raw_data = df[data_cols].values
std_data = StandardScaler().fit_transform(raw_data)
red = umap.UMAP(n_neighbors=10,random_state=55).fit(std_data) #Fixed RNG seed
ids = df["id"]
embedding = red.embedding_ #Was red.fit_transform
embedding_df = pd.DataFrame(data=embedding, index=ids.values, columns=["umap_0", "umap_1"])

## UMAP Diagrams

In [7]:
#Fist UMAP diagram
fig, ax1 = plt.subplots(figsize=(5,4))
fig.tight_layout()
sns.scatterplot(x=embedding_df["umap_0"], y=embedding_df["umap_1"])
plt.draw()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
#Associate the d_x and d_y normalized geodata with the UMAP data
x_min = min(df["latitude"])
y_min = min(df["longitude"])
d_x = df.set_index("id")["latitude"].apply(lambda x: x - x_min)
d_y = df.set_index("id")["longitude"].apply(lambda y: y - y_min)
d_x = d_x / max(d_x)
d_y = d_y / max(d_y)
emb_df = embedding_df.copy()
emb_df["d_x"] = d_x
emb_df["d_y"] = d_y

In [9]:
#Second UMAP diagram, associating UMAP values to point sizes
cmap = sns.cubehelix_palette(light=0.75,as_cmap=True)
fig, ax1 = plt.subplots(figsize=(5,4))
fig.tight_layout()
sns.scatterplot(x=emb_df["umap_0"], y=emb_df["umap_1"], alpha = 0.5, size=100 * emb_df["d_y"], sizes=(0, 100), hue = emb_df["d_x"], palette=cmap)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Marker map

In [10]:
#Monochromatic marker map. Marker size and color denotes UMAP values
#plt.figure(figsize=(25,30))
fig, ax1 = plt.subplots(figsize=(8,8))
fig.tight_layout()
ax1.set_xlim([-0.05,1.05])
ax1.set_ylim([-0.05,1.05])
sns.scatterplot(x=emb_df["d_y"], y=emb_df["d_x"], alpha = 0.5, size=100 * emb_df["umap_1"], sizes=(0, 100), hue = emb_df["umap_0"], palette=cmap)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
#Polychromatic marker map. We use CIELab to associate UMAP data to color ranges
#Ranges: 0,1 for RGB, -128,127 for CIELab
lab_light = 50 #CIELAB L variable
emb_df[['umap_norm_0','umap_norm_1']] = MinMaxScaler(feature_range=(-127,128)).fit_transform(emb_df[['umap_0','umap_1']])
emb_df['umap_color_lab'] = emb_df.apply(lambda x: LabColor(lab_light,x['umap_norm_0'], x['umap_norm_1']), axis=1)
emb_df['umap_color_rgb'] = emb_df.apply(lambda x: convert_color(x['umap_color_lab'], sRGBColor), axis=1) #para poder ser ploteado, hay que convertir a RGB
emb_df['umap_color_rgb'] = emb_df.apply(lambda x: (x['umap_color_rgb'].clamped_rgb_r,x['umap_color_rgb'].clamped_rgb_g,x['umap_color_rgb'].clamped_rgb_b), axis=1) #Se pierde fidelidad al clampear
#plt.figure(figsize=(25,30))
#plt.scatter(emb_df['d_y'],emb_df['d_x'],c = emb_df['umap_color_rgb'],s=90)
fig, ax1 = plt.subplots(figsize=(8,8))
fig.tight_layout()
ax1.set_xlim([-0.05,1.05])
ax1.set_ylim([-0.05,1.05])
emb_df.plot.scatter('d_y','d_x',c = emb_df['umap_color_rgb'],s=12, ax=ax1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
mark_id = 266218 #ID of the reference marker. Currently a known marker that exists in the dataset as default
threshold = 0.14 #Maximum UMAP distance to graph
min_dist = 0 #Minimum geographical distance to graph in the PCA stage

In [13]:
#More widgets
def make_comparisons(btn):
    global mark_id
    mark_id = id_field.value
    display(Javascript('IPython.notebook.execute_cells_below()'))
    
def id_observer(field):
    global mark_id
    mark_id = field.new
    
comp_btn = Button(description="Make comparisons")
comp_btn.on_click(make_comparisons)
id_field = IntText(value=266218, description='Marker ID')
display(comp_btn, id_field)

Button(description='Make comparisons', style=ButtonStyle())

IntText(value=266218, description='Marker ID')

In [14]:
#Calculation of UMAP distances between the reference Marker ID and all other markers
def umap_dist(x_u0,x_u1,y_u0,y_u1):
    return sqrt((x_u0 -  y_u0)**2 + (x_u1 - y_u1)**2)
    
def max_dist(umap_pts): #Input: [(umap_x_1, umap_y_1),...]
    max_dist = 0
    for (x, y) in umap_pts:
        for (w, z) in umap_pts:
            d = umap_dist(x,y,w,z)
            if d > max_dist:
                max_dist = d
    return max_dist

#Similarity between markers as a function of similar CIELAB hue. Deprecated
#mark_colr = emb_df.loc[mark_id,'umap_color_lab']
#emb_df['color_sim'] = emb_df.apply(lambda x: delta_e_cie2000(mark_colr,x['umap_color_lab']), axis=1) #Lento, evitese ejecutar demasiado
mark_u0 = emb_df.loc[mark_id,'umap_0']
mark_u1 = emb_df.loc[mark_id,'umap_1']
maximum = max_dist([(u0,u1) for u0,u1 in zip(emb_df['umap_0'],emb_df['umap_1'])])    #Lento
emb_df['mark_umap_dist'] = emb_df.apply(lambda x: umap_dist(mark_u0,mark_u1,x['umap_0'],x['umap_1'])/maximum, axis=1) #Normalizado

In [15]:
#UMAP distance threshold widget. 
def threshold_observer(field):
    global threshold
    threshold = field.new
    display(Javascript('IPython.notebook.execute_cells_below()'))

threshold_field = FloatText(value=threshold,Min=0,Max=20,description='Maximum UMAP distance')
threshold_field.observe(threshold_observer,'value')
display(threshold_field)

FloatText(value=0.14, description='Maximum UMAP distance')

In [16]:
#Relative change between two markers
def relative_change(x,y):
    return abs(x-y)/((abs(x)+abs(y))/2)

#Extract the group of markers under the UMAP distance threshold, and associate the real geodata to it
#emb_df_sim = emb_df.loc[emb_df['color_sim']<=threshold].copy()
emb_df_sim = emb_df.loc[emb_df['mark_umap_dist']<=threshold].copy()
emb_df_sim['mark_umap_rc'] = emb_df_sim.apply(lambda x: (relative_change(mark_u0,x['umap_0'])+relative_change(mark_u1,x['umap_1']))/2, axis=1) #Percentage of similarity against marked id
df_geo = df[['id','latitude','longitude']].set_index('id')
emb_df_sim = emb_df_sim.join(df_geo,how='inner')
mark = df.loc[df['id']==mark_id]
mark_geo = (df.loc[df['id']==mark_id]['latitude'],df.loc[df['id']==mark_id]['longitude'])
emb_df_sim['mark_geo_dist'] = emb_df_sim.apply(lambda x: lldist(mark_geo[0],mark_geo[1],x['latitude'],x['longitude']), axis=1)
emb_df_sim.to_csv('geodata.csv') #Geodata out, theoretically usable in Hit-Map

In [17]:
#Deprecated graph, superseded by interactive graph below
#plt.figure(figsize=(25,30))
#plt.xlim(0, 1)
#plt.ylim(0, 1)
#plt.scatter(emb_df_sim['d_y'],emb_df_sim['d_x'],c = emb_df_sim['umap_color_rgb'], s = 90)
#plt.show()
rc_pro = emb_df_sim.drop([mark_id])['mark_umap_rc'].mean()*100
print('Average relative change (percentage): ' + str(rc_pro) + '%')

Average relative change (percentage): 49.44338662730131%


In [18]:
#Interactive similarity graph, with on hover info per marker.
#Black star denotes the reference marker, all other markers are under the UMAP distance threshold
label_text = "ID {0}\nGeographical Distance {1}\nUMAP Distance {2}\nRelative Change {3}%"
fig, ax2 = plt.subplots(figsize=(8,8))
fig.tight_layout()
ax2.set_xlim([-0.05,1.05])
ax2.set_ylim([-0.05,1.05])
emb_df_sim.plot.scatter('d_y','d_x',c = emb_df_sim['umap_color_rgb'],s=12, ax=ax2)
mplcursors.cursor(ax2,hover=True).connect("add", lambda sel: sel.annotation.set_text(
    label_text.format(emb_df_sim.iloc[sel.target.index].name,emb_df_sim.iloc[sel.target.index]['mark_geo_dist'],emb_df_sim.iloc[sel.target.index]['mark_umap_dist'],emb_df_sim.iloc[sel.target.index]['mark_umap_rc'])))
ax2.scatter(emb_df_sim.loc[mark_id]['d_y'],emb_df_sim.loc[mark_id]['d_x'],c='k',marker='*')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
#PCA model from subset of markers under the threshold
filter_ids = emb_df_sim.index
df_pca = df[df['id'].isin(filter_ids)].copy().reset_index()
raw_data_pca = df_pca[data_cols].values
std_data_pca = StandardScaler().fit_transform(raw_data_pca)
pca = PCA(n_components=2)
pc_data = pca.fit_transform(std_data_pca)
pc_df = pd.DataFrame(data = pc_data,columns = ['pc_0', 'pc_1'])
sim_data = emb_df_sim[['umap_color_rgb','mark_geo_dist']].reset_index().rename(columns={"index": "id"}) #Restoring the id column from index
pc_df = pd.concat([pc_df, sim_data], axis = 1)

In [20]:
#Widget for minimum geographical distance between the reference Marker ID and other markers in the filtered set.
#Default of 0 for no geographical filtering
def min_dist_observer(field):
    global min_dist
    min_dist = field.new
    display(Javascript('IPython.notebook.execute_cells_below()'))

min_dist_field = FloatText(value=min_dist,Min=0,Max=20,description='Minimum geographical distance')
min_dist_field.observe(min_dist_observer,'value')
display(min_dist_field)

FloatText(value=0.0, description='Minimum geographical distance')

In [21]:
#PCA graph of the reference marker and neighborhood of markers under the UMAP distance threshold.
#Minimum geographical distance optional, no marker is removed from the model
label_text = "ID {0}\nGeographical Distance {1}"
pc_df_filtered = pc_df.loc[pc_df['mark_geo_dist']>=min_dist].copy()
fig, ax3 = plt.subplots(figsize=(8,8))
fig.tight_layout()
#ax3.set_xlim([-6,14])
#ax3.set_ylim([-6,14])
ax3.set_aspect('equal', adjustable='datalim')
pc_df_filtered.plot.scatter('pc_0','pc_1',c = pc_df_filtered['umap_color_rgb'],s=12, ax=ax3)
mplcursors.cursor(ax3,hover=True).connect("add", lambda sel: sel.annotation.set_text(
    label_text.format(pc_df_filtered.iloc[sel.target.index]['id'],pc_df_filtered.iloc[sel.target.index]['mark_geo_dist'])))
ax3.scatter(pc_df[pc_df['id']==mark_id]['pc_0'],pc_df[pc_df['id']==mark_id]['pc_1'],c='k',marker='*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
#PCA values from the model
print("PCA Averages")
{d:m for d,m in zip(data_cols,pca.mean_)}

PCA Averages


{'hm17com_pob_tot': 1.1534784671430199e-17,
 'hm17mz_pob_tot_ssum2': -1.932076432464558e-16,
 'hm17mz_lab_pea_ssum2': 7.930164461608261e-17,
 'hm17mz_gse_ab_ssum2': -8.074349270001139e-17,
 'hm17mz_gse_c1a_ssum2': 7.209240419643874e-17,
 'hm17mz_gse_c1b_ssum2': -1.0092936587501423e-17,
 'hm17mz_gse_c2_ssum2': 2.0906797216967234e-17,
 'hm17mz_gse_c3_ssum2': 1.4418480839287748e-18,
 'hm17mz_gse_d_ssum2': -2.0185873175002847e-17,
 'hm17mz_gse_e_ssum2': -5.767392335715099e-18,
 'Hm17mz_ing_au_2hog_sv2': -3.2297397080004555e-16,
 'Hm17mz_ing_au_2pce_sv2': -1.6220790944198715e-17,
 'pore2_AreasVerdes': -6.344131569286608e-17,
 'pore2_clinica': -3.604620209821937e-17,
 'pore2_estacionesmetro': -7.425517632233189e-17,
 'pore2_hospital': -4.6139138685720794e-17,
 'pore2_pharmacy': -2.883696167857549e-17,
 'pore2_saludeducacion': 3.7488050182148143e-17,
 'pore2_supermarket': -2.3069569342860397e-17,
 'pore2_TiendasConveniencia': 9.083642928751281e-17}

In [23]:
#ID of a target marker for one to one comparisons
target_id = 0

In [24]:
#Widget for the target marker
def target_observer(field):
    global target_id
    target_id = field.new
    display(Javascript('IPython.notebook.execute_cells_below()'))

target_id_field = FloatText(value=target_id,Min=0,Max=1000000,description='Target marker ID')
target_id_field.observe(target_observer,'value')
display(target_id_field)

FloatText(value=0.0, description='Target marker ID')

In [25]:
#Variable per variable comparisons between reference marker and a single target marker
#Comparison methods may need a review
def similarity(x,y):
    t = (x,y)
    return min(t)/max(t) 
try:
    target_index = df[df['id']==target_id][data_cols].index[0]
    mark_index = df[df['id']==mark_id][data_cols].index[0]
    rows = df.loc[[mark_index,target_index], data_cols]
    rc_values = rows.apply(lambda x: str(similarity(x[mark_index],x[target_index])*100)+'%')
    print(rc_values.rename(desc_all))
except IndexError:
    print('Select a valid target mark ID')

Select a valid target mark ID


In [26]:
#Additional CSS for dashboard layout
%%html
<style>
.grid-stack-item.ui-draggable.ui-resizable{
    border: 3px double black
}
.widget-checkbox{
    display:block !important;
    width: auto;
}
.grid-stack.grid-stack-animate, .grid-stack.grid-stack-animate .grid-stack-item{
    background-color: lightgray; !important
}
</style>

SyntaxError: invalid syntax (<ipython-input-26-0346e874dd44>, line 3)